## clinical variable retrieval codebase.
each querey is referencing a sql querey linked in my github for ALL patients in the database, then generating a dataframe, then paring that dataframe down to only the patients/icustay_id in our cohort. 
* 5-16-19 heavily streamlined, can now change global variables at top of page which will correspond to all variables. added all code into functions and made a composite function to run each variable. 
* each variable is also deleted to reduce rolling memory usage


### overall workflow of this notebook:
1) extract vitals first

2) limit patients to those with appropriate vitals going forward. (final_pt_df_v)

3) extract all other clinical variables 
 * after extracting all patients first, filter to those patints in minimum vitals list
 * save and delete after extracting to reduce unnescessary memory load

_note_: the sql scripts are currently setup to drop a materialized view if exists and create a new materialized view with every run.
 one could setup this workflow so that these views are only created once by removing this part in each sql script, or simply querying the view name instead.

In [1]:
#import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import collections
import asyncio
import getpass
import re
from datetime import datetime as dt
import os,sys,re
from collections import Counter
import seaborn as sns
import random
from pathlib import Path

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

In [2]:
# note, all server information is stored in a config.py file that is present in the .gitignore
import config 
conn = psycopg2.connect(dbname=config.dbname, user=config.user, host=config.host, port=config.port,password=config.password)
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'


time: 59.7 ms


In [3]:
from parameters import repository_path
os.chdir(repository_path)
wd= os.getcwd()

time: 1.13 s


In [12]:
#patients of interest from rotation_cohort_generation
from parameters import final_pt_df, date, repository_path

#patients of interest from rotation_cohort_generation
final_pt_df2 = final_pt_df
del(final_pt_df)

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 112 ms


In [5]:
# # run if already have a final_pt_df_v stored (which is ensuring pts have adequate vitals to be considered suspected of BI)
# from parameters import final_pt_df_v
# final_pt_df2= final_pt_df_v
# icustay_id_vitals= list(final_pt_df_v['icustay_id'].unique())

time: 697 µs


In [6]:
final_pt_df2['final_bin'].value_counts()

C_neg/A_partial    7966
C_neg/A_full       7504
C_pos/A_full       2478
C_pos/A_partial    1971
Name: final_bin, dtype: int64

time: 8.83 ms


## Access MIMIC database and convert it to dataframe in Pandas

In [7]:
#input the sql_exe_show object and get dataframe for only patients in patient list out. 
def sql_exe_show(sql_sentence):
    cur.execute(sql_sentence)
    rows = cur.fetchall()
    col = []
    for i in range(len(cur.description)):
        col.append(cur.description[i][0])
    table = pd.DataFrame(rows,columns=col)
    return table

def sql_to_df_icu(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['icustay_id'].isin(icustay_id)]
    return sql_exe_show_df

def sql_to_df_patients(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['subject_id'].isin(patients)]
    return sql_exe_show_df

def sql_to_df_hadm(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['hadm_id'].isin(hadm_id)]
    return sql_exe_show_df

def clinvar_fxn(var_name, path, subject_id_override=False):
    f= open(path, 'r')
    var = f.read()
    cur.execute('rollback')
    cur.execute(var)
    
    if subject_id_override==True:
        df= sql_to_df_patients(sql_exe_show('select * from %s;' %(var_name)))
    
    else:
        
        try:
            df= sql_to_df_icu(sql_exe_show('select * from %s;' %(var_name)))
        except KeyError or NameError:
            try:      
                df= sql_to_df_hadm(sql_exe_show('select * from %s;' %(var_name)))
            except KeyError or NameError:
                df= sql_to_df_patients(sql_exe_show('select * from %s;' %(var_name)))

    print(df.shape)
    return(df)


time: 74.2 ms


# extracting clinical data for our patients
* clinical data window= (t0+x)- t0+y
 * lower_window: x, set this to offset the t_0 for lower bound of the clinical time window
 * upper_window: y, set this to set the upper bound of the clinical time window.
* folder: folder name for data to be stored in
* date: date attached in file name of all files associated with this data
* time_col: the time column used to restrict data to the clinical data window.
* patient_df: the cohort dataframe used, default: final_pt_df2
* save_boolean: # do you want to save the files generated?
    * if yes, save_boolean=True
    * else save_boolean=False

In [8]:
from parameters import lower_window, upper_window, folder, date, time_col, time_var, patient_df, save_boolean

time: 948 µs


# important functions

In [18]:
def time_window_filter(df, final_pt_df2,timecol,upper_window, lower_window, time_var='t_0'):
    """
    will take in any df and filter to only values between lower_window and upper_window. 
    values = the difference between timecol-timevar. 
        most commonly this is charttime- t_0 (or first in ICU ABrx meeting criteria)
    will add delta and t_0 to df as well which are used lateron.git
    """
    #global upper_window, lower_window 
    
    try:
        df= pd.merge(df, final_pt_df2[['icustay_id',time_var]], left_on= 'icustay_id', right_on = 'icustay_id') #n=240317
        df['delta']= pd.to_datetime(df[timecol]) - pd.to_datetime(df[time_var])
        df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
        df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=upper_window),:]
    except KeyError or NameError:
        df= pd.merge(df, final_pt_df2[['hadm_id',time_var]], left_on= 'hadm_id', right_on = 'hadm_id') #n=240317
        df['delta']= pd.to_datetime(df[timecol]) - pd.to_datetime(df[time_var])
        df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
        df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=upper_window),:]
    return(df_after_t0)

time: 22.5 ms


# variable extraction below:

## Vital Sign

In [ ]:
##-- This query pivots the vital signs for the first 24 hours of a patient's stay
##-- Vital signs include heart rate, blood pressure, respiration rate, and temperature

vitals_all_nosummary_df= clinvar_fxn(
    'vitals_all_nosummary',
    str(repository_path)+'/src/SQL/vitals_all_nosummary.sql'
    )

In [24]:
##-- This query pivots the vital signs for the first 24 hours of a patient's stay
##-- Vital signs include heart rate, blood pressure, respiration rate, and temperature

vitals_all_nosummary_df= clinvar_fxn(
    'vitals_all_nosummary',
    str(repository_path)+'/src/SQL/vitals_all_nosummary.sql'
    )

(22340510, 7)
time: 17min


In [9]:
# trying to select the data from the materialized view
# vitals_all_nosummary_sql = query_schema + """
# SELECT *
# FROM public.vitals_all_nosummary
# """

# #admissions
# vitals_all_nosummary_df=pd.read_sql_query(vitals_all_nosummary_sql,conn)
# vitals_all_nosummary_df= vitals_all_nosummary_df[vitals_all_nosummary_df['icustay_id'].isin(icustay_id)]

time: 14min 14s


In [25]:
vitals_all_nosummary_df.head()

,subject_id,hadm_id,icustay_id,charttime,valueuom,vitalid,valuenum
3,9,150750,220597,2149-11-11 00:00:00,mmHg,MeanArtPress,81.0
4,9,150750,220597,2149-11-11 19:00:00,mmHg,MeanArtPress,72.0
5,9,150750,220597,2149-11-11 19:45:00,mmHg,MeanArtPress,76.0
6,9,150750,220597,2149-11-11 20:00:00,mmHg,MeanArtPress,99.0
9,21,111970,216859,2135-02-01 15:30:00,mmHg,MeanArtPress,61.0


time: 172 ms


# filtering to patients with bare minimum vital numbers
previously we found that 3% or so of patients don't have baseline vitals counts. this is filtering the patients to only those who have this baseline value.

In [19]:
#using origional criteria to find pts who have atleast 1 spo2 reading within 3 days of t_0

#The idea is that this should be the bare minimum amount of data for a patient, and without it, it's likely the physicians did not suspect an infection in these patients. 
##NOTE: this should not change when the clinical timewindow of analysis interest changes. 

vitals_filter = time_window_filter(vitals_all_nosummary_df, final_pt_df2, time_col,time_var='t_0', lower_window=0,upper_window=3 )

vitals_filter= vitals_filter.loc[
    vitals_filter['vitalid'].notnull(),:]

icustay_id_vitals = (vitals_filter.loc[
    vitals_filter.loc[:,'vitalid']=='SpO2','icustay_id'
        ].unique())

subject_id_vitals=list(final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_vitals),'subject_id'])
hadm_id_vitals= list(final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_vitals),'hadm_id'])
icustay_id_vitals= list(final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_vitals),'icustay_id'])
del(vitals_filter)

##saving the patient database and reassigning patient set to the patient set with minimum vitals

final_pt_df2_v=final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_vitals),:]
if save_boolean==True:
    #pd.DataFrame(final_pt_df2_v).to_csv("/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_final_pt_df2_v.csv" %date) #final cohort database n=11493 subject_id’s (7/6/18)
    pd.DataFrame(final_pt_df2_v).to_csv((str(repository_path)+ '/data/{}_final_pt_df_v.csv'.format(date)), index=False)
    final_pt_df2=final_pt_df2_v.copy()


time: 25.7 s


In [20]:
final_pt_df2=final_pt_df2_v
final_pt_df2_v['final_bin'].value_counts()

C_neg/A_partial    7867
C_neg/A_full       7401
C_pos/A_full       2438
C_pos/A_partial    1927
Name: final_bin, dtype: int64

time: 22.5 ms


## filtering, subset, and composite functions to be used for the rest of the notebook

In [62]:
def df_subset(df):
    """
    redundancy check to ensure all df are filtered to cohort with minimum vitals 
    """
    try:
        df = df.loc[df.loc[:,'icustay_id'].isin(icustay_id_vitals),:]
    except KeyError or NameError:
        try:
            df = df.loc[df.loc[:,'hadm_id'].isin(hadm_id_vitals),:]
        except KeyError or NameError:
            df = df.loc[df.loc[:,'subject_id'].isin(subject_id_vitals),:]
    return(df)

time: 5.22 ms


In [30]:
def filter_subset_save(df, savename=None, return_df=False, save=False, time_filter_override=False):
    
    """
    composite function, performs 1: time_window_filter() and 2:df_subset() to the input dataframe. this function links them together for simplifying code needed after each sql and formatting query. 
    return_df specifies if any output is spit out.
    save specifies if the file will be saved with teh savename.
    
    fxn was created on 5/16/19 and validated against the normal pipeline.
    """
    
    global date,folder,final_pt_df2, lower_window, upper_window, time_var, timecol, time_var

    if time_filter_override==False:
        time_filtered= time_window_filter(df, final_pt_df2, timecol=time_col ,time_var=time_var, lower_window=lower_window, upper_window=upper_window)
    else:
        time_filtered=df
        
    time_and_subseted= df_subset(time_filtered)
    
    if save==True:
        #os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling')
        if folder != None:
             address=str(repository_path)+'/data/raw/%s/'%(folder)
        else:
            address = str(repository_path)+'/data/raw/'
        if not os.path.exists(address):
            print(address)
            os.makedirs(address)
            
        pd.DataFrame(time_and_subseted).to_csv(address+'/%s_%s.csv' %(date, savename))       
    else: pass
    
    if return_df==False:
        del(df, time_filtered, time_and_subseted)
    
    else:
        return(time_and_subseted)
    

time: 103 ms


In [31]:
filter_subset_save(vitals_all_nosummary_df, savename="vitals_all_nosummary", save=save_boolean, return_df=False)

/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/48_hr_window/
time: 3min 44s


In [32]:
del(vitals_all_nosummary_df)

time: 18.7 ms


## elixhauser comobridities

In [33]:
elixhauser_nosummary_df= clinvar_fxn(
    'elixhauser_quan',
    str(repository_path)+'/src/SQL/elixhauser_quan.sql', 
    subject_id_override=True
)

(26825, 33)
time: 1min 4s


### now next task: redo same as above, BUT EXCLUDE CURRENT ROW FROM CUMMAX()

In [34]:
def elix(shift=False):
    elix_var=list(elixhauser_nosummary_df)[3:]
    elixhauser_nosummary_df2=elixhauser_nosummary_df.copy()

    elixhauser_nosummary_df2[elix_var]=(elixhauser_nosummary_df
                                            .sort_values('stay_num', ascending=True) #sorts values so stay_num is ascending
                                            .groupby('subject_id', as_index=False)[elix_var] #groups by subject id and filters only elixhauser variable columns
                                            .agg('cummax') #takes a cummulitive max for every row
                                       )

    if shift==True:
        #now shifting the values up by 1 so the cumulitive max doesn't consider the current values: (note: couldn't get this to work in the fxn above)
        elixhauser_nosummary_df2[elix_var]=(elixhauser_nosummary_df
                                                .sort_values('stay_num', ascending=True) #sorts values so stay_num is ascending
                                                .groupby('subject_id', as_index=False)[elix_var] #groups by subject id and filters only elixhauser variable columns
                                                .shift(fill_value=np.nan)[:-1]) #shifts the cummulitive max up by 1 so the first row is na. 

    #restricting to hadm in use
    elixhauser_nosummary_df3= elixhauser_nosummary_df2[elixhauser_nosummary_df2['hadm_id'].isin(hadm_id)]
    
    #adding icustay_id
    elixhauser_nosummary_df3=pd.merge(elixhauser_nosummary_df3, final_pt_df2[['hadm_id','icustay_id']], how="left", left_on='hadm_id',right_on='hadm_id')
    
    return(elixhauser_nosummary_df3)

time: 31.6 ms


In [35]:
elixhauser_df=elix(shift=False)
cancer_elix=elixhauser_df[['subject_id','hadm_id','icustay_id']].copy()
cancer_elix['value']=elixhauser_df.loc[:,['lymphoma',"solid_tumor","metastatic_cancer"]].max(axis=1)
#adding columns
cancer_elix['label']= 'cancer_elix'
cancer_elix['delta']= 0
cancer_elix['delta']= pd.to_timedelta(cancer_elix['delta'], unit='d')
cancer_elix['uom']= 'pos/neg category'

filter_subset_save(cancer_elix, savename="cancer_elix", save=save_boolean, return_df=False,time_filter_override=True) #filtering to ppl with sufficient vitals
del(elixhauser_df,cancer_elix)

elixhauser_df=elix(shift=True)
elix_var=list(elixhauser_df)[3:-1]
sum_elix=elixhauser_df[['subject_id','hadm_id','icustay_id']].copy()
sum_elix['value']=elixhauser_df.loc[:,elix_var].sum(axis=1)
#adding columns
sum_elix['label']= 'sum_elix'
sum_elix['delta']= 0
sum_elix['delta']= pd.to_timedelta(sum_elix['delta'], unit='d')
sum_elix['uom']= 'elixhauser_comorb_sum'

filter_subset_save(sum_elix, savename="sum_elix", save=save_boolean, return_df=False,time_filter_override=True) #filtering to ppl with sufficient vitals
del(elixhauser_df,sum_elix)

time: 1.45 s


# demographics

In [39]:
def demographics():
    """
    wrapping demographics code into a fxn. basically combines ethinicity, age, gender and race into one df. 
    """
    global final_pt_df2
    pt_info_sql = query_schema + """
    SELECT SUBJECT_ID, INSURANCE, LANGUAGE, RELIGION, MARITAL_STATUS, ETHNICITY
    from mimiciii.admissions
    ORDER BY subject_id DESC
    """

    pt_info_df=pd.read_sql_query(pt_info_sql,conn) #361711 patients with sterile culture -> 374643 with addn of bal and broncho... 7/16/18
    
    ethnicity_df=(pt_info_df.loc[
    pt_info_df.loc[:,"subject_id"].isin(
        final_pt_df2['subject_id'].tolist()),:]).drop_duplicates(['subject_id','ethnicity'])
    ethnicity_df= ethnicity_df[['subject_id','ethnicity']].sort_values('ethnicity', ascending=False).groupby('subject_id', as_index=False).first()
    
    #gender and age
    pt_info_sql = query_schema + """
    SELECT icu.icustay_id,
        icu.subject_id,
        (extract( epoch from icu.intime-pd.dob))/60/60/24/365.25 as age,
        pd.gender
    FROM  mimiciii.icustays icu
    LEFT JOIN mimiciii.patients pd
        ON pd.subject_id = icu.subject_id
    """
    
    #admissions
    pt_info_df=pd.read_sql_query(pt_info_sql,conn) #361711 patients with sterile culture -> 374643 with addn of bal and broncho... 7/16/18
    
    #combining gender, race
    pt_info_df=(pt_info_df.loc[
        pt_info_df.loc[:,"subject_id"].isin(
            final_pt_df2['subject_id'].tolist()),:]).drop_duplicates(['subject_id','gender'])
    pt_info_df=(pt_info_df.loc[pt_info_df.loc[:,"icustay_id"].isin(icustay_id),:])
    pt_info_df= pd.merge(pt_info_df, ethnicity_df, left_on='subject_id', right_on='subject_id', how='left' )

    #combining age, gender and race. 
    pt_info_df=pd.merge(pt_info_df, final_pt_df2[['icustay_id','t_0']])

    pt_info_df[pt_info_df['age']>89]['age']=90
    
    age_df=pd.melt(pt_info_df, id_vars=['icustay_id','subject_id','t_0'])
    age_df=age_df.rename(index=str, columns={'variable':'label'})

    age_df['delta']=pd.to_timedelta('0 days')
    age_df['uom']="N/A"
    age_df.loc[age_df.loc[:,'label']=='first_admit_age','uom']='years'

    age_df= age_df.loc[age_df.loc[:,"icustay_id"].isin(icustay_id),:]

    ###using regular expressions to reduce the # of ethinicities
    age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'.*?BLACK')),'value']="black"
    age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'.*?HISPANIC|PORTUGUESE')),'value']="hispanic"
    age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'.*?WHITE')),'value']="white/nonhispanic"
    age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'.*?ASIAN')),'value']='asian'
    age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'(UNKNOWN|MULTI|UNABLE|DECLINE|OTHER)')),'value']='unknown/other'
    age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'[AZ]+')),'value']="unknown/other" #lumping all other low n values into other
    return(age_df)


time: 112 ms


In [40]:
age_df= demographics()

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


time: 2.75 s


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:60: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [41]:
age_df.head()

,icustay_id,subject_id,t_0,label,value,delta,uom
0,280836,268,2198-02-16,age,65.9849,0 days,N/A
1,206613,269,2170-11-05,age,40.0998,0 days,N/A
2,219649,275,2170-10-08,age,82.1642,0 days,N/A
3,204407,279,2164-06-15,age,74.294,0 days,N/A
4,257572,281,2101-10-18,age,60.0142,0 days,N/A


time: 9.43 ms


In [42]:
filter_subset_save(age_df, savename="pt_info", save=save_boolean, return_df=False, time_filter_override=True)
del(age_df)

time: 717 ms


## Weight

In [45]:
#prereq to weight and height
echo_df= clinvar_fxn(
    'echodata',
    str(repository_path)+'/src/SQL/echodata.sql' 
)

(15893, 18)
time: 37.4 s


In [46]:
# -- This query extracts weights for adult ICU patients on their first ICU day.
# -- It does *not* use any information after the first ICU day, as weight is
# -- sometimes used to monitor fluid balance.

weightfirstday_df= clinvar_fxn(
    'weightfirstday',
    str(repository_path)+'/src/SQL/weightfirstday.sql' 
)
weightfirstday_df['uom']='kg'

(19919, 6)
time: 14.6 s


In [47]:
filter_subset_save(weightfirstday_df, savename="weightfirstday", save=save_boolean, return_df=False, time_filter_override=True)
del(weightfirstday_df)

time: 148 ms


## Height

In [48]:
# -- This query extracts heights for adult ICU patients.
# -- It uses all information from the patient's first ICU day.
# -- This is done for consistency with other queries - it's not necessarily needed.
# -- Height is unlikely to change throughout a patient's stay.
heightfirstday_df= clinvar_fxn(
    'heightfirstday',
    str(repository_path)+'/src/SQL/heightfirstday.sql' 
)

heightfirstday_df['uom']='cm'

(19919, 4)
time: 5.85 s


In [49]:
filter_subset_save(heightfirstday_df, savename="heightfirstday", save=save_boolean, return_df=False, time_filter_override=True)
del(heightfirstday_df)

time: 110 ms


In [50]:
del(echo_df)

time: 8.61 ms


# Labs

In [51]:
# -- This query pivots lab values for all patients, then filtered to those in my cohort.
# -- Have confirmed that the unit of measurement is always the same: null or the correct unit

labs_all_nosummary_df= clinvar_fxn(
    'labs_all_nosummary',
    str(repository_path)+'/src/SQL/labs_all_nosummary.sql'
)

(5673182, 6)
time: 3min 39s


In [52]:
#importing unit of mesurements:
def uom_sql_import(file_path):
    if isinstance(file_path, str)== True:
        f = open(Path(file_path), 'r')
    else:
        f = open(Path(str(file_path)), 'r')

    SQL = open(file_path,'r').read()
    SQL_df= pd.read_sql_query(SQL,conn)   
    return(SQL_df)

time: 40.8 ms


In [53]:
lab_uom= uom_sql_import(Path(str(repository_path)+'/src/SQL/labs_uom.sql'))
labs_all_nosummary_df = pd.merge(labs_all_nosummary_df, lab_uom, left_on='label', right_on='label')

labs_all_nosummary_df[labs_all_nosummary_df['label']=='LYMPHO%']#.value_counts() #4-15-19: what is this, is this exploring absolute lymphocyte %?
#labs_all_nosummary_df['label'].value_counts()

labs_all_nosummary_df[labs_all_nosummary_df['label']=='WBC']['valuenum'].describe()
#labs_all_nosummary_df[labs_all_nosummary_df['label']=='PLATELET']['valuenum'].describe()


count    334371.000000
mean         11.596203
std           8.901332
min           0.090000
25%           6.900000
50%          10.100000
75%          14.300000
max         471.700000
Name: valuenum, dtype: float64

time: 27.5 s


In [54]:
filter_subset_save(labs_all_nosummary_df, savename="labs_all_nosummary", save=save_boolean, return_df=False, time_filter_override=False)
del(labs_all_nosummary_df)
del(lab_uom)

time: 28.5 s


## Glasgow Coma Scale

In [55]:
#modified on 8/28/18 to have the days annotation.
##--8/28/18: added in epoch as days, in order to help determine btwn t_0 and 72 hour for pts.
gcsall_days_df= clinvar_fxn(
    'gcsall_days', 
    str(repository_path)+'/src/SQL/gcsall_days.sql' 
)

(19919, 9)
time: 1min 29s


In [56]:
#adding in icu_admit time and filtereing time_var to time window.
def gcs_72(gcsall_days_df,final_pt_df2, time_var='t_0', lower_window=0, upper_window=3):
    """
    written a while back, aims to adding in icu_admit time and filtereing time_var to time window.
    will use this with time_filter_override=True in my filter_subset_save()
    """
    
    ##merging gcsall_days_df with final_pt df in order to append on icustay_id, the time var, and ICU_admit
    gcsall_days_df_merge= pd.merge(
    gcsall_days_df,
    final_pt_df2[['icustay_id','icu_admit',time_var]],
    left_on='icustay_id',
    right_on='icustay_id')

    gcsall_days_df_merge['day'] = gcsall_days_df_merge['day']-1 #putting the epoch days so that 0 = the first day
    
    #approximating the charttime of the time associated with each gcs score
    gcsall_days_df_merge['approx_charttime']=pd.to_timedelta((gcsall_days_df_merge['day'])*24, unit='h') + pd.to_datetime(gcsall_days_df_merge['icu_admit'])
    
    # day # + ICU_admission day.
    gcsall_days_df_merge['admit_plus_day']= (
        pd.to_datetime(gcsall_days_df_merge['icu_admit'])
        + pd.to_timedelta(gcsall_days_df_merge['day'], unit='D')
    )
    
    #difference between the admission+epoch day - time_var.
    gcsall_days_df_merge['delta']= (
        pd.to_datetime(gcsall_days_df_merge['admit_plus_day']) - pd.to_datetime(gcsall_days_df_merge[time_var])
    )
    
    #filtering day windows
    gcsall_days_df_merge_72= (
        gcsall_days_df_merge.loc[gcsall_days_df_merge.loc[:,'delta']>= pd.Timedelta(days=lower_window),:])
    gcsall_days_df_merge_72= (
        gcsall_days_df_merge_72.loc[gcsall_days_df_merge_72.loc[:,'delta']<= pd.Timedelta(days=upper_window),:])
    return(gcsall_days_df_merge_72)

time: 29.9 ms


In [57]:
gcs72_df = gcs_72(gcsall_days_df,final_pt_df2, time_var=time_var, lower_window=lower_window,upper_window=upper_window )
gcs72_df['uom']='GCS_score' #adding in uom


time: 758 ms


In [58]:
filter_subset_save(gcs72_df, savename="gcs", save=save_boolean, return_df=False, time_filter_override=True)
del(gcs72_df)

time: 219 ms


## Renal replacement therapy

the sql code for this was not equipped to join all of the charttimes together. so i decided to do it in python below.
the rrt_all_df code above was only a 1 or 0 if patient had RRT during their entire icu stay. 

- step 1: run all sql codes
- 2: filter on only the t_0 to t_72 hour rows
- 3: filter on the 1223 patients who have a positive value
- 4: get the earliest incidence of rrt for each 1223 patients. 

In [59]:
def rrt_runmerge():
    """
    wrapping a lot of scripting into a function. grabs the 5 different rrt datas, filters them to timewindow, and merges them into 1 dataframe.
    """
    global date,folder, patient_df,lower_window, upper_window, time_var, time_var, time_col
    
    ###5 sql queries to grab raw data
    
    #mv_ce
    f = open(str(repository_path)+'/src/SQL/rtt_mv_ce.sql' , 'r')  
    rrtSQL_mv_ce = f.read()
    rrtSQL_mv_ce_sql = query_schema + rrtSQL_mv_ce.format(tuple(patients))
    rrtSQL_mv_ce_df=pd.read_sql_query(rrtSQL_mv_ce_sql,conn)    
    #cv
    f = open(str(repository_path)+'/src/SQL/rtt_cv.sql' , 'r') 
    rrtSQL_cv = f.read()
    rrtSQL_cv_sql = query_schema + rrtSQL_cv.format(tuple(patients))
    rrtSQL_cv_df=pd.read_sql_query(rrtSQL_cv_sql,conn)          
    #mv_ie
    f = open(str(repository_path)+'/src/SQL/rtt_mv_ie.sql' , 'r') 
    rrtSQL_mv_ie = f.read()
    rrtSQL_mv_ie_sql = query_schema + rrtSQL_mv_ie.format(tuple(patients))
    rrtSQL_mv_ie_df=pd.read_sql_query(rrtSQL_mv_ie_sql,conn)      
    rrtSQL_mv_ie_df['charttime']= rrtSQL_mv_ie_df['starttime']
    rrtSQL_mv_ie_df=rrtSQL_mv_ie_df.drop('starttime', axis=1)
    #mv_de
    f = open(str(repository_path)+'/src/SQL/rtt_mv_de.sql' , 'r') 
    rrtSQL_mv_de = f.read()
    rrtSQL_mv_de_sql = query_schema + rrtSQL_mv_de.format(tuple(patients))
    rrtSQL_mv_de_df=pd.read_sql_query(rrtSQL_mv_de_sql,conn)      
    #mv_pe
    f = open(str(repository_path)+'/src/SQL/rtt_mv_pe.sql' , 'r') 
    rrtSQL_mv_pe = f.read()
    rrtSQL_mv_pe_sql = query_schema + rrtSQL_mv_pe.format(tuple(patients))
    rrtSQL_mv_pe_df=pd.read_sql_query(rrtSQL_mv_pe_sql,conn)          
    rrtSQL_mv_pe_df['charttime']= rrtSQL_mv_pe_df['starttime']
    rrtSQL_mv_pe_df=rrtSQL_mv_pe_df.drop('starttime', axis=1)
    
    ### timewindow filtering

    def hour_72_window_rrt(df, final_pt_df2,timecol='charttime',time_var='t_0', lower_window=0, upper_window=3 ):
        ##modified to make more generalizable to easily accomidate PA cohort but default to my origional cohort.
        ##filters rrt to within timewindow  between timecol- time_var

        df= pd.merge(final_pt_df2[['icustay_id',time_var]], df, left_on= 'icustay_id', right_on = 'icustay_id', how='left') #n=240317
        df['delta']= pd.to_datetime(df[timecol]) - pd.to_datetime(df[time_var])
        df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
        df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=upper_window),:] 
        #df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'rrt']==1,:].groupby('icustay_id')['charttime'].min()
        return(pd.DataFrame(df_after_t0))#.reset_index())
    
    rrtSQL_mv_ce_pt =hour_72_window_rrt(rrtSQL_mv_ce_df, patient_df, timecol=time_col,time_var=time_var, lower_window=lower_window,upper_window=upper_window)
    rrtSQL_cv_pt =hour_72_window_rrt(rrtSQL_cv_df, patient_df, timecol=time_col,time_var=time_var, lower_window=lower_window,upper_window=upper_window)
    rrtSQL_mv_ie_pt =hour_72_window_rrt(rrtSQL_mv_ie_df, patient_df, timecol=time_col,time_var=time_var, lower_window=lower_window,upper_window=upper_window)
    rrtSQL_mv_de_pt =hour_72_window_rrt(rrtSQL_mv_de_df, patient_df, timecol=time_col,time_var=time_var, lower_window=lower_window,upper_window=upper_window)
    rrtSQL_mv_pe_pt =hour_72_window_rrt(rrtSQL_mv_pe_df, patient_df, timecol=time_col,time_var=time_var, lower_window=lower_window,upper_window=upper_window)
    
    ### merging all 5 filtered rrt_df together
    
    def rrt_merging(rrtSQL_mv_ce_pt, rrtSQL_cv_pt, rrtSQL_mv_ie_pt, rrtSQL_mv_de_pt, rrtSQL_mv_pe_pt, timecol='charttime',time_var='t_0'):
        ###returns an aggregate y/n of if patient had positive rrt within timewindow. 

        rrt_merged_pt= pd.concat([rrtSQL_mv_ce_pt, rrtSQL_cv_pt, rrtSQL_mv_ie_pt, rrtSQL_mv_de_pt, rrtSQL_mv_pe_pt])

        #making a 1 if has positive rrt within timewindow:
        rrt_merged_pt= pd.DataFrame(rrt_merged_pt.loc[rrt_merged_pt.loc[:,'rrt']==1,:].groupby('icustay_id')[timecol].min().reset_index())
        rrt_merged_pt['rrt']=1

        rrt_merged_allpt_df= pd.merge(final_pt_df2[['icustay_id',time_var]], rrt_merged_pt, left_on= 'icustay_id', right_on = 'icustay_id', how='left') #n=240317
        rrt_merged_allpt_df=rrt_merged_allpt_df.rename(index=str, columns={timecol:"first_charttime"})

        rrt_merged_allpt_df['uom']='category' #adding a uom category
        rrt_merged_allpt_df.loc[rrt_merged_allpt_df.loc[:,'rrt'].isnull(),'rrt']='0'

        return(rrt_merged_allpt_df)


    rrt_merged_allpt_df= rrt_merging(rrtSQL_mv_ce_pt, rrtSQL_cv_pt, rrtSQL_mv_ie_pt, rrtSQL_mv_de_pt, rrtSQL_mv_pe_pt, timecol=time_col,time_var=time_var)
    return(rrt_merged_allpt_df)

time: 205 ms


In [60]:
rrt_merged_allpt_df= rrt_runmerge()
filter_subset_save(rrt_merged_allpt_df, savename="rrt_merged", save=save_boolean, return_df=False, time_filter_override=True)
del(rrt_merged_allpt_df)

time: 29.2 s


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:63: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



# UTI related variables

In [64]:
uti_all_df= clinvar_fxn(
    'uti_all',
    str(repository_path)+'/src/SQL/gcsall_days.sql' 
)

(1109441, 12)
time: 2min 13s


In [65]:
filter_subset_save(uti_all_df, savename="uti_all", save=save_boolean, return_df=False, time_filter_override=False)
del(uti_all_df)

time: 9.61 s


## Blood Gas Test

In [66]:
def PaO2(bg_all_nosummary_df):
    """
    overview: replaces the PO2 label with PaO2 on all instances (defined as sharing icustay_id and charttime being equal)
        where the specimen label == 'ART'
        
    input: bloodgas dataframe with values annotated. 
    output: bloodgas dataframe with values annotated where PO2 label is replaced with PaO2 according to above criteria
    """
    
    #making a unique varaible to search for and mark all rows where time and icustay_id has an art flag
    bg_all_nosummary_df['unique_var']= bg_all_nosummary_df['icustay_id'].map(str) +  bg_all_nosummary_df['charttime'].map(str)

    #making subset dataframe for label == SPECIMEN 
    bg_all_nosummary_specimen= bg_all_nosummary_df.loc[bg_all_nosummary_df.loc[:,'label']=='SPECIMEN',:]
    
    #all ART related rows: unique_var for all rows where label== SPECIMEN
    bg_all_nosummary_ART = bg_all_nosummary_specimen[bg_all_nosummary_specimen['value']=='ART'] 
    bg_all_nosummary_ART_list= list(bg_all_nosummary_ART['unique_var'].unique())

    #two criteria needed to change the PO2 to PaO2 label.
    criteria1=(bg_all_nosummary_df['label'] == 'PO2')
    criteria2=(bg_all_nosummary_df['unique_var'].isin(bg_all_nosummary_ART_list))
    
    #making changes
    bg_all_nosummary_df.loc[(criteria2 & criteria1),'label']= 'PaO2'
    
    return(bg_all_nosummary_df)

time: 32.7 ms


In [67]:
bg_all_nosummary_df= clinvar_fxn(
    'bg_all_nosummary',
    str(repository_path)+'/src/SQL/bg_all_nosummary.sql' 
    ) 
bg_all_nosummary_df = PaO2(bg_all_nosummary_df)


(2774898, 8)
time: 2min 19s


In [68]:
filter_subset_save(bg_all_nosummary_df, savename="bg_all_nosummary", save=save_boolean, return_df=False, time_filter_override=False)
del(bg_all_nosummary_df)

time: 22 s


## Vaso_active therapies

In [69]:
# 10/12/18 added amountuom as amount_uom, rateuom as rate_uom to many lines of the sql code.

weightdurations_df= clinvar_fxn(
    'weightdurations',
    str(repository_path)+'/src/SQL/weightdurations.sql'  ##added to vasoactive_meds due to dependency of SQL code
)
#
epi_dose_df= clinvar_fxn(
    'epinephrine_dose',
    str(repository_path)+'/src/SQL/epinephrine_dose.sql' 
)
#
norepi_dose_df= clinvar_fxn(
    'norepinephrine_dose',
    str(repository_path)+'/src/SQL/norepinephrine_dose.sql' 
)
#
dopamine_dose_df= clinvar_fxn(
    'dopamine_dose',
    str(repository_path)+'/src/SQL/dopamine_dose.sql' 
)
#
dobutamine_dose_df= clinvar_fxn(
    'dobutamine_dose',
    str(repository_path)+'/src/SQL/dobutamine_dose.sql' 
)
#
vasopressin_dose_df= clinvar_fxn(
    'vasopressin_dose',
    str(repository_path)+'/src/SQL/vasopressin_dose.sql' 
)

#removing units/hour, as these are not appropriate
vasopressin_dose_df= vasopressin_dose_df.loc[~vasopressin_dose_df.loc[:,'rate_uom'].isin(['Uhr','units/hour']),:]
#
phenylephrine_dose_df= clinvar_fxn(
    'phenylephrine_dose',
    str(repository_path)+'/src/SQL/phenylephrine_dose.sql' 
)

(71511, 4)
(5793, 7)
(71889, 7)
(15274, 7)
(4757, 7)
(15270, 7)
(51967, 7)
time: 1min 22s


In [70]:
#adding an identification label column and merging them into 1 df. 
epi_dose_df['label']='epinephrine'
norepi_dose_df['label']='norepinephrine'
dopamine_dose_df['label']='dopamine'
dobutamine_dose_df['label']='dobutamine'
vasopressin_dose_df['label']='vasopressin'
phenylephrine_dose_df['label']='phenylephrine'
vaso_dose_df = pd.concat([epi_dose_df, norepi_dose_df, dopamine_dose_df, dobutamine_dose_df, vasopressin_dose_df,phenylephrine_dose_df ])

#rename starttime to charttime
vaso_dose_df.rename(index=str, columns={'starttime':"charttime"}, inplace=True)

time: 448 ms


In [71]:
filter_subset_save(vaso_dose_df, savename="vaso_dose", save=save_boolean, return_df=False, time_filter_override=False)
del(vaso_dose_df)
del(epi_dose_df, norepi_dose_df, dopamine_dose_df, dobutamine_dose_df, vasopressin_dose_df,phenylephrine_dose_df)

time: 1.54 s


# ventilator settings and categorization

In [72]:
#ventsettings_df = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/src/csv/15082018_ventsettings_df.csv', index_col=0)
ventsettings_df= clinvar_fxn(
    'ventsettings',
    str(repository_path)+'/src/SQL/ventsettings.sql' 
)
#going from wide format to long:
#pd.melt(ventsettings_df, id_vars=['icustay_id','charttime'])

(1078506, 6)
time: 3min 26s


In [73]:
def vent_data(vent_df,time_var='t_0', lower_window=0, upper_window=3 ):
    df= pd.merge(vent_df,
                final_pt_df2[['icustay_id',time_var]],
                left_on='icustay_id',
                right_on= 'icustay_id',
                how='left')
    df['delta']= pd.to_datetime(df['charttime']) - pd.to_datetime(df[time_var])
    df_timewindow= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
    df_timewindow= df_timewindow.loc[df_timewindow.loc[:,'delta']<= pd.Timedelta(days=upper_window),:] 
    df_timewindow['day']= df_timewindow['delta'].apply(lambda x: pd.to_timedelta(x,unit='d').days) #day #
      
    return(df_timewindow)

#df_timewindow =vent_data(ventsettings_df,time_var='first_pos_else_neg_ssc', lower_window=-1, upper_window=1 )

def vent_day_categorizer(vent_df,time_var='t_0', lower_window=0, upper_window=3 ):
    
    df_timewindow =vent_data(vent_df,time_var=time_var, lower_window=lower_window, upper_window=upper_window)
    
    df_timewindow_perday=df_timewindow.groupby(['icustay_id','day'], as_index=False)[['mechvent','oxygentherapy']].agg({'mechvent':'max', 'oxygentherapy':'max'})

    conditions= [
        (df_timewindow_perday['mechvent']==1),
        ((df_timewindow_perday['oxygentherapy']==1) & (df_timewindow_perday['mechvent']==0)),
        (df_timewindow_perday['oxygentherapy']==0 & (df_timewindow_perday['mechvent']==0))]

    choices=['Mech', 'Oxygen', 'None']
    #
    df_timewindow_perday['value']= np.select(conditions, choices, default='error:no_value_filled')
    df_timewindow_perday['value']
    df_timewindow_perday=df_timewindow_perday.reset_index()
    df_timewindow_perday['uom']= 'mech/O2/none category'
    df_timewindow_perday= df_timewindow_perday.drop(['mechvent','oxygentherapy','index'], axis=1)
    df_timewindow_perday=pd.merge(df_timewindow_perday, final_pt_df2[['icustay_id',time_var]] )
    return(df_timewindow_perday)


time: 53.9 ms


In [74]:
#ventcategory_df = vent_categorization(final_pt_df2, ventsettings_df, time_var='first_pos_else_neg_ssc' )
ventcategory_df= vent_day_categorizer(ventsettings_df,time_var=time_var, lower_window=lower_window, upper_window=upper_window)
#ventcount_df = vent_count(final_pt_df2,ventsettings_df, time_var='first_pos_else_neg_ssc')

time: 5.14 s


In [75]:
filter_subset_save(ventcategory_df, savename="ventcategory", save=save_boolean, return_df=False, time_filter_override=True)
#del(ventcategory_df)

time: 251 ms


In [76]:
ventcategory_df.head()

,icustay_id,day,value,uom,t_0
0,200001.0,0,Oxygen,mech/O2/none category,2181-11-26
1,200001.0,1,Oxygen,mech/O2/none category,2181-11-26
2,200003.0,0,Mech,mech/O2/none category,2199-08-04
3,200003.0,1,Mech,mech/O2/none category,2199-08-04
4,200012.0,0,Oxygen,mech/O2/none category,2153-12-23


time: 27.2 ms


# daily SOFA score

### running yiheng's sql codes to capture daily sofa_score. ideally i coulda just used my data above but she had this written already so i'll use this.

link to her github: https://github.com/yihengpan/fluid_management/tree/master/sofa

Here is how we generate the sofa table:


sofa <- scorecalc <- scorecomp <- vaso_cv    <-icu_18
                                             
                                             <-wt      <-icu_18
                                             <-echo2
                                          
                               <- vaso_mv    <-icu_18
                               <- pafi2      <-bloodgas_pan_aterial <- bloodgas_pan  <-icu_18
                                             <-ventelurations
                               <- vitals_pan <-icu_18
                               <- labs_pan   <-icu_18
                               <- uo_pan     <-icu_18
                               <- gcs_pan    <-icu_18



In [77]:
sofa_path= str(repository_path)+'/src/SQL/sofa'
var='wt'
wt_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='echo2'
echo2_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='bloodgas_pan'
bloodgas_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='bloodgas_pan_arterial'
bloodgas_pan_art_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

(33300, 3)
(19919, 2)
(294454, 33)
(241035, 39)
time: 3min 23s


In [78]:
#vaso_mv, vaso_cv,pafi2, vitals_pan, labs_pan, uo_pan, gcs_pan
sofa_path= str(repository_path)+'/src/SQL/sofa'
var='vaso_mv'
vaso_mv_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='vaso_mv'
vaso_mv_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='vaso_cv'
vaso_cv_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='pafi1'
pafi1_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='pafi2'
pafi2_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)


var='labs_pan'
labs_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='uo_pan'
uo_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='gcs_pan'
gcs_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

###

var='scorecomp'
scorecomp_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='scorecalc'
scorecalc_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='sofa_pan'
sofa_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

(8209, 6)
(8209, 6)
(11257, 6)
(241035, 5)
(76566, 4)
(116400, 42)
(111927, 5)
(19692, 9)
(133916, 14)
(133916, 20)
(125866, 23)
time: 8min 19s


In [79]:
final_pt_df2.head()

,icustay_id,ssc_charttime,ssc_id,icu_admit,ab_id,ab_start,ab_end,ab_ssc_delta,t_0,t_end,...,subject_id,dod_hosp,dod_ssn,ab_course,org_list,spec_type_list,first_pos_else_neg_ssc,sc_result,final_bin,hadm_id
0,294638,2191-03-16 00:00:00,22,2191-03-16 00:29:31,1213116,2191-03-16,2191-03-22,0 days 00:00:00.000000000,2191-03-16,2191-03-22,...,4,NaN,NaN,full,STAPH AUREUS COAG +,BLOOD CULTURE,2191-03-16 00:00:00,positive,C_pos/A_full,185777
1,220597,2149-11-10 09:40:00,45,2149-11-09 13:07:02,2089126,2149-11-10,2149-11-15,0 days 00:00:00.000000000,2149-11-10,2149-11-15,...,9,2149-11-14,2149-11-14,full,NaN,NaN,2149-11-10 09:40:00,negative,C_neg/A_full,150750
2,232669,2104-08-11 00:00:00,60,2104-08-08 02:08:17,616189,2104-08-11,2104-08-12,0 days 00:00:00.000000000,2104-08-11,2104-08-12,...,12,2104-08-20,2104-08-20,partial,NaN,NaN,2104-08-11 00:00:00,negative,C_neg/A_partial,112213
3,273430,2108-08-05 20:42:00,69,2108-08-05 16:26:09,2572274,2108-08-06,2108-08-08,1 days 00:00:00.000000000,2108-08-06,2108-08-08,...,19,NaN,2109-08-18,partial,NaN,NaN,2108-08-05 20:42:00,negative,C_neg/A_partial,109235
4,217847,2134-09-11 09:35:00,70,2134-09-11 20:50:04,1388217,2134-09-12,2134-09-13,1 days 00:00:00.000000000,2134-09-12,2134-09-13,...,21,2135-02-08,2135-02-08,full,NaN,NaN,2134-09-11 09:35:00,negative,C_neg/A_full,109451


time: 37.5 ms


In [80]:
sofa_pan_df.head()

,subject_id,hadm_id,icustay_id,day,sofa,respiration,pao2fio2_vent_min,pao2fio2_novent_min,coagulation,platelet_min,...,rate_dopamine,rate_epinephrine,rate_norepinephrine,rate_dobutamine,meanbp_min,cns,mingcs,renal,creatinine_max,urineoutput
6,4,185777,294638,1.0,2,NaN,NaN,NaN,0.0,201.0,...,NaN,NaN,NaN,NaN,69.0,0.0,15.0,0.0,0.5,2150.0
7,4,185777,294638,2.0,0,NaN,NaN,NaN,0.0,258.0,...,NaN,NaN,NaN,NaN,93.0,NaN,NaN,0.0,0.4,900.0
12,9,150750,220597,1.0,5,3.0,125.0,NaN,0.0,249.0,...,NaN,NaN,NaN,NaN,67.0,NaN,NaN,1.0,1.4,887.0
13,9,150750,220597,2.0,5,3.0,150.0,NaN,0.0,221.0,...,NaN,NaN,NaN,NaN,73.0,0.0,15.0,2.0,2.0,2628.0
14,9,150750,220597,3.0,5,3.0,170.0,NaN,0.0,261.0,...,NaN,NaN,NaN,NaN,72.0,NaN,NaN,2.0,2.0,2330.0


time: 22.1 ms


In [81]:
#deleting these to clear up memory 

del(vaso_cv_df, vaso_mv_df, labs_pan_df, gcs_pan_df, scorecalc_df, scorecomp_df, uo_pan_df, pafi1_df, pafi2_df, bloodgas_pan_art_df, echo2_df, wt_df)

time: 42.2 ms


In [82]:
print(
sofa_pan_df['hadm_id'].nunique(), #8707
final_pt_df2['hadm_id'].nunique() #8731
)

18998 18773
time: 4.18 ms


In [83]:
print(
sofa_pan_df['icustay_id'].nunique(), #8707
final_pt_df2['icustay_id'].nunique() #8731
)

19884 19626
time: 3.12 ms


In [84]:
sofa_pan_df['hadm_id'].nunique() #8707
final_pt_df2['hadm_id'].nunique() #8731

18773

time: 4.26 ms


In [85]:
#adding in t_0 & icuadmit date
def sofa_day_window_filter(sofa_pan_df, time_var= 't_0', lower_window= 0, upper_window=3): #'t_0'):
    import datetime
    '''
    #Yihangpan wrote a sql script and materialized view "sofa_pan" which gives the sofa score for each day in icu for each patient. 
    #since the sofa_pan has days after admission but not chartdates, I need to use day # to find the associated t_0 to t_0+72 for each patient. 
    # the challenge was that I had to relate day# in sofa_pan to my t_0 date. the day # was based on the days after icu admission, where day1 = the first day (day=0)= 0 to 24 hours post admission.
    #To do this, I added day# (where day 0 is the first day) to icu admission date. 
    #I then filtered on only the rows where this icuadmin + day# was between t_0 and t_0 + 72 hours. 

    #since t_0 has only day resolution, and for that I ignored hours and only took the date (rounded down all hours/minutes/seconds). this is similar to how i made the t_0 date. 
    #the problem this creates is that it widens the potential time window, so it theoretically can contain up to 95.99 hours, since hours on day 1 were collapsed to 0. 

    
    input: 
        sofa_pan_df: daily sofa scores captured from sofa_pan_sql.
        optional:
            time_var: the time variable we want to base the window off of
            window_bottom= 0, time_var- window_bottom (days + time_var) = first daily sofa score to capture
            window_top= 0, time_var- window_top (days + time_var) = last daily sofa score to capture 
    
    output: sofa_pan_sql annotated with days and filtered to time window set by window_bottom and window_top. 
    '''
    #reducing to minimum vital patients
    sofa_pan_df=sofa_pan_df.loc[sofa_pan_df.loc[:,"icustay_id"].isin(icustay_id_vitals),:].copy()

    ##merging sofa_pan with final_pt df in order to append on icustay_id, the time var, and ICU_admit
    
    sofa_df_merged= pd.merge(sofa_pan_df,
                             final_pt_df2[['icustay_id',time_var,'icu_admit']],
                             left_on= 'icustay_id',
                             right_on = 'icustay_id',
                             how='left') #n=240317

    #sofa_df_merged['admit_t0_rounded'] = pd.to_datetime(sofa_df_merged['ICU_admit']).dt.round('1440min')
    
    sofa_df_merged['day'] = sofa_df_merged['day']-1 #putting the epoch days so that 0 = the first day
    
    #approximating the charttime of the time associated with each sofa score. adding on days to icuadmit. 
    sofa_df_merged['approx_charttime']=pd.to_timedelta((sofa_df_merged['day'])*24, unit='h') + pd.to_datetime(sofa_df_merged['icu_admit'])

    #rounding down the charttime to the day, so hours and minutes are ignored (just like t_0)
    sofa_df_merged['floor_charttime'] = sofa_df_merged['approx_charttime'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, 24*(dt.hour//24))) 
    
    sofa_df_merged['floor_time_var'] = pd.to_datetime(sofa_df_merged[time_var]).apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, 24*(dt.hour//24))) 
    
    sofa_df_window= sofa_df_merged.loc[
        (sofa_df_merged['floor_charttime'].between(
            (pd.to_datetime(sofa_df_merged['floor_time_var'])+ pd.to_timedelta(lower_window, unit='d')),
            (pd.to_datetime(sofa_df_merged['floor_time_var'])+ pd.to_timedelta(upper_window, unit='d')+ pd.to_timedelta(1, unit='h')) #added 1hr timebuffer incase between is set as less than greater than
        )),:]
    
    return(sofa_df_window)
    #return(sofa_df_window.drop(['floor_time_var','floor_charttime'], axis=1))


time: 39.4 ms


In [89]:
sofa_df_window= sofa_day_window_filter(sofa_pan_df, time_var= time_var, lower_window= lower_window, upper_window=upper_window)
sofa_df_window.sort_values(['icustay_id','day','sofa','rate_dopamine','rate_epinephrine','rate_norepinephrine','rate_dobutamine'],
                          ascending=[True, True, False, False, False, False, False], inplace=True
                          )
sofa_df_window.drop_duplicates(['icustay_id','day','sofa'], inplace=True)
print(len(sofa_df_window))

filter_subset_save(sofa_df_window, savename="sofa", save=save_boolean, return_df=False, time_filter_override=True)


43631
time: 2.37 s


# end of extraction